# Importing Libraries

In [1]:
from tensorflow.keras.applications import VGG16
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Importing Open Pose

MPII Output Format: Head – 0 Neck – 1 Right Shoulder – 2 Right Elbow – 3 Right Wrist – 4 Left Shoulder – 5 Left Elbow – 6 Left Wrist – 7 Right Hip – 8 Right Knee – 9 Right Ankle – 10 Left Hip – 11 Left Knee – 12 Left Ankle – 13 Chest – 14 Background – 15

In [2]:
MODE = "MPI"

if MODE is "COCO":
    protoFile = "pose/coco/pose_deploy_linevec.prototxt"
    weightsFile = "pose/coco/pose_iter_440000.caffemodel"
    nPoints = 18
    POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

elif MODE is "MPI" :
    protoFile = "pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
    weightsFile = "pose/mpi/pose_iter_160000.caffemodel"
    nPoints = 15
    POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

inWidth = 368
inHeight = 368

In [3]:
base_dir_op = 'DataBaseCustom/DATASET'
sub_dir_op = os.listdir(base_dir_op)
temp = []
for sd in sub_dir_op:
    temp.append(os.path.join(base_dir_op, sd))
sub_dir_op = temp 
print(sub_dir_op)

['DataBaseCustom/DATASET\\TEST', 'DataBaseCustom/DATASET\\TRAIN']


In [4]:
sub_dir_poses = os.listdir(sub_dir_op[0])
print(sub_dir_poses)

['goddess', 'halfMoon', 'tree', 'triangle', 'warrior2']


In [5]:
train_poses_paths = []
test_poses_paths = []
for pose in sub_dir_poses:
    train_poses_paths.append(os.path.join(sub_dir_op[1], pose))
    test_poses_paths.append(os.path.join(sub_dir_op[0], pose))
print(train_poses_paths)

['DataBaseCustom/DATASET\\TRAIN\\goddess', 'DataBaseCustom/DATASET\\TRAIN\\halfMoon', 'DataBaseCustom/DATASET\\TRAIN\\tree', 'DataBaseCustom/DATASET\\TRAIN\\triangle', 'DataBaseCustom/DATASET\\TRAIN\\warrior2']


In [6]:
train_img_paths = []
test_img_paths = []
for path in train_poses_paths:
    img_dir = os.listdir(path)
    temp = []
    for img in img_dir:
        temp.append(os.path.join(path, img))
    train_img_paths.append(temp)
for path in test_poses_paths:
    img_dir = os.listdir(path)
    temp = []
    for img in img_dir:
        temp.append(os.path.join(path, img))
    test_img_paths.append(temp)
print(len(train_img_paths))

5


In [9]:
def create_db():
    bd1 = 'DataBaseCustom/DATASET_BLACK'
    bd2 = 'DataBaseCustom/DATASET_OP'
    test_path_1 = os.path.join(bd1, 'TEST')
    train_path_1 = os.path.join(bd1, 'TRAIN')
    test_path_2 = os.path.join(bd2, 'TEST')
    train_path_2 = os.path.join(bd2, 'TRAIN')
    os.mkdir(test_path_1)
    os.mkdir(train_path_1)
    os.mkdir(test_path_2)
    os.mkdir(train_path_2)
    sub_dir_1 = os.listdir(bd1)
    sub_dir_2 = os.listdir(bd2)
    for pose in sub_dir_poses:
        os.mkdir(os.path.join(test_path_1, pose))
        os.mkdir(os.path.join(train_path_1, pose))
        os.mkdir(os.path.join(test_path_2, pose))
        os.mkdir(os.path.join(train_path_2, pose))    
    temp = []
    for sd in sub_dir_1:
        temp.append(os.path.join(bd1, sd))
    sub_dir_1 = temp
    temp = []
    for sd in sub_dir_2:
        temp.append(os.path.join(bd2, sd))
    sub_dir_2 = temp
    print("sub_dir_1: ", sub_dir_1)
    print("sub_dir_2: ", sub_dir_2)
    train_paths_1 = []
    test_paths_1 = []
    train_paths_2 = []
    test_paths_2 = []
    for pose in sub_dir_poses:
        train_paths_1.append(os.path.join(sub_dir_1[1], pose))
        test_paths_1.append(os.path.join(sub_dir_1[0], pose))
        train_paths_2.append(os.path.join(sub_dir_2[1], pose))
        test_paths_2.append(os.path.join(sub_dir_2[0], pose))
    print("train_paths_1: ", train_paths_1)
    print("train_paths_2: ", train_paths_2)
    total_img_count = 0
    print("Begin Training Transfer: ")
    for count, pose in enumerate(train_img_paths):
        count2 = 0
        for img in pose:       

            inWidth = 368
            inHeight = 368

            frame = cv2.imread(img)
            frameWidth = frame.shape[1]
            frameHeight = frame.shape[0]
            threshold = 0.1

            black_canvas = np.zeros(shape = (frameHeight, frameWidth, 3))

            inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                      (0, 0, 0), swapRB=False, crop=False)

            net.setInput(inpBlob)

            output = net.forward()
            H = output.shape[2]
            W = output.shape[3]

            # Empty list to store the detected keypoints
            points = []

            for i in range(nPoints):
                # confidence map of corresponding body's part.
                probMap = output[0, i, :, :]

                # Find global maxima of the probMap.
                minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

                # Scale the point to fit on the original image
                x = (frameWidth * point[0]) / W
                y = (frameHeight * point[1]) / H

                if prob > threshold : 
                    cv2.circle(frame, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
                    cv2.circle(black_canvas, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
                    # Add the point to the list if the probability is greater than the threshold
                    points.append((int(x), int(y)))
                else :
                    points.append(None)

            # Draw Skeleton
            for pair in POSE_PAIRS:
                partA = pair[0]
                partB = pair[1]

                if points[partA] and points[partB]:
                    cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3)
                    cv2.line(black_canvas, points[partA], points[partB], (0, 255, 255), 3)
            image_name = sub_dir_poses[count] + '_' + str(count2) + '.jpg'
            count2 = count2 + 1
            print("image_count = ", total_img_count)
            total_img_count = total_img_count + 1
            temp_path_1 = os.path.join(train_paths_1[count], image_name)
            cv2.imwrite(temp_path_1, black_canvas)
            temp_path_2 = os.path.join(train_paths_2[count], image_name)
            cv2.imwrite(temp_path_2, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    print("Begin Testing Transfer: ")
    for count, pose in enumerate(test_img_paths):
        count2 = 0
        for img in pose:       

            inWidth = 368
            inHeight = 368

            frame = cv2.imread(img)
            frameWidth = frame.shape[1]
            frameHeight = frame.shape[0]
            threshold = 0.1

            black_canvas = np.zeros(shape = (frameHeight, frameWidth, 3))

            inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                      (0, 0, 0), swapRB=False, crop=False)

            net.setInput(inpBlob)

            output = net.forward()
            H = output.shape[2]
            W = output.shape[3]

            # Empty list to store the detected keypoints
            points = []

            for i in range(nPoints):
                # confidence map of corresponding body's part.
                probMap = output[0, i, :, :]

                # Find global maxima of the probMap.
                minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

                # Scale the point to fit on the original image
                x = (frameWidth * point[0]) / W
                y = (frameHeight * point[1]) / H

                if prob > threshold : 
                    cv2.circle(frame, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
                    cv2.circle(black_canvas, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
                    # Add the point to the list if the probability is greater than the threshold
                    points.append((int(x), int(y)))
                else :
                    points.append(None)

            # Draw Skeleton
            for pair in POSE_PAIRS:
                partA = pair[0]
                partB = pair[1]

                if points[partA] and points[partB]:
                    cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3)
                    cv2.line(black_canvas, points[partA], points[partB], (0, 255, 255), 3)
            image_name = sub_dir_poses[count] + '_' + str(count2) + '.jpg'
            count2 = count2 + 1
            print("image_count = ", total_img_count)
            total_img_count = total_img_count + 1
            temp_path_1 = os.path.join(test_paths_1[count], image_name)
            cv2.imwrite(temp_path_1, black_canvas)
            temp_path_2 = os.path.join(test_paths_2[count], image_name)
            cv2.imwrite(temp_path_2, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [10]:
create_db()

sub_dir_1:  ['DataBaseCustom/DATASET_BLACK\\TEST', 'DataBaseCustom/DATASET_BLACK\\TRAIN']
sub_dir_2:  ['DataBaseCustom/DATASET_OP\\TEST', 'DataBaseCustom/DATASET_OP\\TRAIN']
train_paths_1:  ['DataBaseCustom/DATASET_BLACK\\TRAIN\\goddess', 'DataBaseCustom/DATASET_BLACK\\TRAIN\\halfMoon', 'DataBaseCustom/DATASET_BLACK\\TRAIN\\tree', 'DataBaseCustom/DATASET_BLACK\\TRAIN\\triangle', 'DataBaseCustom/DATASET_BLACK\\TRAIN\\warrior2']
train_paths_2:  ['DataBaseCustom/DATASET_OP\\TRAIN\\goddess', 'DataBaseCustom/DATASET_OP\\TRAIN\\halfMoon', 'DataBaseCustom/DATASET_OP\\TRAIN\\tree', 'DataBaseCustom/DATASET_OP\\TRAIN\\triangle', 'DataBaseCustom/DATASET_OP\\TRAIN\\warrior2']
Begin Training Transfer: 
image_count =  0
image_count =  1
image_count =  2
image_count =  3
image_count =  4
image_count =  5
image_count =  6
image_count =  7
image_count =  8
image_count =  9
image_count =  10
image_count =  11
image_count =  12
image_count =  13
image_count =  14
image_count =  15
image_count =  16
image

image_count =  401
image_count =  402
image_count =  403
image_count =  404
image_count =  405
image_count =  406
image_count =  407
image_count =  408
image_count =  409
image_count =  410
image_count =  411
image_count =  412
image_count =  413
image_count =  414
image_count =  415
image_count =  416
image_count =  417
image_count =  418
image_count =  419
image_count =  420
image_count =  421
image_count =  422
image_count =  423
image_count =  424
image_count =  425
image_count =  426
image_count =  427
image_count =  428
image_count =  429
image_count =  430
image_count =  431
image_count =  432
image_count =  433
image_count =  434
image_count =  435
image_count =  436
image_count =  437
image_count =  438
image_count =  439
image_count =  440
image_count =  441
image_count =  442
image_count =  443
image_count =  444
image_count =  445
image_count =  446
image_count =  447
image_count =  448
image_count =  449
image_count =  450
image_count =  451
image_count =  452
image_count 

image_count =  833
image_count =  834
image_count =  835
image_count =  836
image_count =  837
image_count =  838
image_count =  839
image_count =  840
image_count =  841
image_count =  842
image_count =  843
image_count =  844
image_count =  845
image_count =  846
image_count =  847
image_count =  848
image_count =  849
image_count =  850
image_count =  851
image_count =  852
image_count =  853
image_count =  854
image_count =  855
image_count =  856
image_count =  857
image_count =  858
image_count =  859
image_count =  860
image_count =  861
image_count =  862
image_count =  863
image_count =  864
image_count =  865
image_count =  866
image_count =  867
image_count =  868
image_count =  869
image_count =  870
image_count =  871
image_count =  872
image_count =  873
image_count =  874
image_count =  875
image_count =  876
image_count =  877
image_count =  878
image_count =  879
image_count =  880
image_count =  881
image_count =  882
image_count =  883
image_count =  884
image_count 

image_count =  1250
image_count =  1251
image_count =  1252
image_count =  1253
image_count =  1254
image_count =  1255
image_count =  1256
image_count =  1257
image_count =  1258
image_count =  1259
image_count =  1260
image_count =  1261
image_count =  1262
image_count =  1263
image_count =  1264
image_count =  1265
image_count =  1266
image_count =  1267
image_count =  1268
image_count =  1269
image_count =  1270
image_count =  1271
image_count =  1272
image_count =  1273
image_count =  1274
image_count =  1275
image_count =  1276
image_count =  1277
image_count =  1278
image_count =  1279
image_count =  1280
image_count =  1281
image_count =  1282
image_count =  1283
image_count =  1284
image_count =  1285
image_count =  1286
image_count =  1287
image_count =  1288
image_count =  1289
image_count =  1290
image_count =  1291
image_count =  1292
image_count =  1293
image_count =  1294
image_count =  1295
image_count =  1296
image_count =  1297
image_count =  1298
image_count =  1299
